In [1]:
import os
from IPython.display import Audio, display, Markdown, HTML
from scipy.io import wavfile

# --- Configuración ---
TEMP_DIR = 'audios_filtrados' # MODIFICACIÓN REALIZADA AQUÍ (Usando el valor que habías solicitado anteriormente)
AUDIO_EXTENSION = '.wav'
FS = 0 # Frecuencia de muestreo (se leerá del primer archivo encontrado)

# Definición del orden de presentación (ancho de banda y tipo de filtro)
BANDAS_ANCHO = ['500Hz', '250Hz', '100Hz', '50Hz']
FILTROS_BASE = ['FIR Kaiser', 'FIR Parks-McClellan', 'IIR Chebyshev I', 'IIR Cauer/Elíptico']

# =========================================================================
# Función Principal para la Disposición HTML
# =========================================================================

def generate_and_display_audio_grid(directory):
    """
    Escanea archivos WAV y genera una estructura HTML con CSS para mostrar 
    los reproductores agrupados por filtro y ordenados por BW.
    """
    if not os.path.isdir(directory):
        display(Markdown(f"## ❌ Error: La carpeta `{directory}` no fue encontrada."))
        return

    # 1. Preparar la lectura de la Frecuencia de Muestreo (FS)
    global FS
    if FS == 0:
        try:
            # Intentar leer FS del archivo 'Original.wav'
            fs_temp, _ = wavfile.read(os.path.join(directory, 'Original.wav'))
            FS = fs_temp
        except Exception:
            # Fallback en caso de error
            display(Markdown("⚠️ **Advertencia:** No se pudo leer la Frecuencia de Muestreo. Se usará 44100 Hz."))
            FS = 44100
    
    # 2. Iniciar el contenido HTML con estilos CSS para la rejilla
    html_content = f"""
    <style>
        .audio-grid-container {{ font-family: Arial, sans-serif; }}
        h2 {{ color: #0056b3; border-bottom: 2px solid #ddd; padding-bottom: 5px; }}
        h4 {{ color: #333; margin-top: 15px; }}
        .original-player-box {{ padding: 10px; border: 1px solid #ccc; background-color: #e6f7ff; border-radius: 5px; margin-bottom: 20px; }}
        
        .filter-group {{ border: 1px solid #0056b3; border-radius: 8px; margin-bottom: 25px; padding: 10px; background-color: #f9f9f9; }}
        .filter-title {{ background-color: #0056b3; color: white; padding: 5px 10px; border-radius: 5px 5px 0 0; margin: -10px -10px 10px -10px; font-weight: bold; }}
        
        .audio-row {{ display: flex; justify-content: space-around; }}
        .audio-col {{ flex: 1; min-width: 24%; padding: 5px; text-align: center; box-sizing: border-box; }}
        .audio-col:not(:last-child) {{ border-right: 1px solid #eee; }}
        
        .bw-label {{ font-size: 0.8em; color: #555; margin-bottom: 5px; font-weight: bold; }}
        audio {{ width: 95%; max-width: 200px; }}
    </style>
    
    <div class="audio-grid-container">
        <h2>🎧 Reproductores de Audio de Señales Filtradas</h2>
    """
    
    # --- 3. Reproductor de Audio Original ---
    original_path = os.path.join(directory, 'Original.wav')
    if os.path.exists(original_path):
        html_content += f"""
        <div class="original-player-box">
            <h4>Señal Original (CW)</h4>
            <audio controls src="{original_path}" type="audio/wav">Su navegador no soporta el audio.</audio>
        </div>
        """
    else:
        html_content += f"<p style='color: red;'>❌ Archivo Original.wav no encontrado.</p>"

    # --- 4. Reproductores Agrupados por Filtro ---
    
    for base_name in FILTROS_BASE:
        html_content += f"""
        <div class="filter-group">
            <div class="filter-title">Filtro: {base_name}</div>
            <div class="audio-row">
        """
        
        for bw_key in BANDAS_ANCHO:
            # Crear el nombre de archivo limpio y la ruta
            safe_name = f"{bw_key}_{base_name}".replace(" ", "_").replace("(", "").replace(")", "").replace("=", "").replace("/", "_")
            filename = os.path.join(directory, f"{safe_name}{AUDIO_EXTENSION}")
            
            # Etiqueta de la columna
            label = f"BW = {bw_key}"
            if bw_key == '50Hz':
                label += " (Selectividad Máx.)"

            # Contenido de la columna
            if os.path.exists(filename):
                html_content += f"""
                <div class="audio-col">
                    <div class="bw-label">{label}</div>
                    <audio controls src="{filename}" type="audio/wav">Su navegador no soporta el audio.</audio>
                </div>
                """
            else:
                html_content += f"""
                <div class="audio-col">
                    <div class="bw-label">{label}</div>
                    <p style="color: red; font-size: 0.8em;">Archivo no encontrado</p>
                </div>
                """
        
        html_content += """
            </div> </div> """
    
    html_content += "</div> "
    
    # Mostrar el HTML generado en el Notebook
    display(HTML(html_content))


# =========================================================================
# Ejecución
# =========================================================================
generate_and_display_audio_grid(TEMP_DIR)

## ❌ Error: La carpeta `audios_filtrados` no fue encontrada.